**instalando PySpark e importando Bibliotecas**

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
import pandas as pd     

In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [ ]:
esquema = ["id", "tipo", "titulo", "país", "data_adicionado" ,"ano_lancamento","classificacao","duracao","genero"]
netflix_df = (spark
       .read
       .format("csv")
       .option("header", "true")
       .option("inferschema", "false")
       .option("delimiter", ",")
       .load("/content/netflix_tratado.csv").toDF(*esquema)
       )

In [ ]:
spark

**Exibindo o esquema do dataframe**

In [ ]:
netflix_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- titulo: string (nullable = true)
 |-- país: string (nullable = true)
 |-- data_adicionado: string (nullable = true)
 |-- ano_lancamento: string (nullable = true)
 |-- classificacao: string (nullable = true)
 |-- duracao: string (nullable = true)
 |-- genero: string (nullable = true)



In [ ]:
netflix_df

DataFrame[id: string, tipo: string, titulo: string, país: string, data_adicionado: string, ano_lancamento: string, classificacao: string, duracao: string, genero: string]


 **Quantidade de títulos disponibilizados pela netflix**

In [ ]:
netflix_df.groupby(F.col("tipo")).count()

DataFrame[tipo: string, count: bigint]

**Separamos os dados que dizem respeito ao público Infantil**

In [ ]:
dfinfantil = netflix_df.filter(F.col('classificacao')=='Infantil')

**Separamos os dados que dizem respeito ao público Adulto**

In [ ]:
dfadultos = netflix_df.filter(F.col('classificacao')=='Adultos')

**Mostra quantos lançamentos teve em cada ano, Demonstrando um aumento grande na produção ou disponibilização de novos filmes.**

In [ ]:
netflix_df.groupBy('ano_lancamento').count().sort('count', ascending=False)

DataFrame[ano_lancamento: string, count: bigint]

In [ ]:
w1 = Window.partitionBy(F.col('ano_lancamento')).orderBy('ano_lancamento')
netflix_df.groupby(F.col('ano_lancamento')).count()
netflix_df.withColumn('rank', F.rank().over(w1))

DataFrame[id: string, tipo: string, titulo: string, país: string, data_adicionado: string, ano_lancamento: string, classificacao: string, duracao: string, genero: string, rank: int]

**Mostra quantos Filmes Series a netflix disponibilizou em sua plataforma nos últimos anos.**

In [ ]:
netflix_df.groupBy('data_adicionado').count().sort('count', ascending=False)

DataFrame[data_adicionado: string, count: bigint]

**Aumento significativo nos ultimos anos**

In [ ]:
netflix_df.filter(F.col('ano_lancamento').like('2018')).count()

1147

In [ ]:
netflix_df.filter(F.col('ano_lancamento').like('2019')).count()

1030

In [ ]:
netflix_df.filter(F.col('ano_lancamento').like('2020')).count()

953

In [ ]:
netflix_df.filter(F.col('ano_lancamento').like('2021')).count()

592

**Exibindo o título e generos apenas dos Filmes**

In [ ]:
netflix_df.select(F.col("ano_lancamento"),F.col("titulo"),F.col("tipo"),F.col("genero")).filter("tipo = 'Filmes'")

DataFrame[ano_lancamento: string, titulo: string, tipo: string, genero: string]

**Exibindo o título e generos apenas das Series**

In [ ]:
netflix_df.select(F.col("ano_lancamento"),F.col("titulo"),F.col("tipo"),F.col("genero")).filter("tipo = 'Séries'")

DataFrame[ano_lancamento: string, titulo: string, tipo: string, genero: string]

**adicionamos uma coluna tempo para analisar a duração dos conteúdos mais assistidos**

In [ ]:
from pyspark.sql.functions import regexp_replace
netflix_df.withColumn('tempo', regexp_replace('duracao', 'min', '')).show(truncate=False)

+---+------+---------------------------------------------------+------------------------------------------------------------------+---------------+--------------+-------------+------------+----------------------------------------------------------------------------------+------------+
|id |tipo  |titulo                                             |país                                                              |data_adicionado|ano_lancamento|classificacao|duracao     |genero                                                                            |tempo       |
+---+------+---------------------------------------------------+------------------------------------------------------------------+---------------+--------------+-------------+------------+----------------------------------------------------------------------------------+------------+
|s1 |Filmes|Dick Johnson Is Dead                               |Estados Unidos                                                    |2021-09-25 

**criando uma coluna Para identificar a plataforma netflix para fazermos junção á plataforma concorrente**

In [ ]:
valorN = 'Netflix'
netflix_df = netflix_df.withColumn("aplicativo", F.lit(valorN))

**Convertendo arquivo em pandas e salvando em CSV**

In [ ]:
netflix_conv =netflix_df.toPandas()

In [ ]:
netflix_conv.to_csv('netflix_trat.csv',index=False)

# Criando uma nova tabela para consulta

In [ ]:
import pandas as pd

In [ ]:
netflix_df = pd.read_csv('/content/netflix_trat.csv')
amazon_df = pd.read_csv('/content/Amazon_trat.csv')

Criando a tabela resultante da junção de dados da Amazon e da Netflix

In [ ]:
amazon_netflix = pd.concat([amazon_df, netflix_df])
amazon_netflix

,id,tipo,titulo,país,data_adicionado,ano_lancamento,classificacao,duracao,genero,aplicativo
0,s1,Filme,A Grande Sedução,Canada,2021-03-30,2014,Nao avaliado,113 min,"Comedia, Drama",Amazon
1,s2,Filme,Cuide da boa noite,India,2021-03-30,2018,13+,110 min,"Drama, Internacional",Amazon
2,s3,Filme,Segredos da Decepção,Estados Unidos,2021-03-30,2017,Nao avaliado,74 min,"Acao, Drama, Suspense",Amazon
3,s4,Filme,Rosa: Mantendo-se Verdadeiro,Estados Unidos,2021-03-30,2014,Nao avaliado,69 min,Documentario,Amazon
4,s5,Filme,Criador de Monstros,Reino Unido,2021-03-30,1989,Nao avaliado,45 min,"Drama, Fantasia",Amazon
...,...,...,...,...,...,...,...,...,...,...
8801,s8803,Filmes,Zodiac,Estados Unidos,2019-11-20,2007,Restrito,158 min,"Filmes de culto, dramas, suspense",Netflix
8802,s8804,Séries,Zombie Dumb,Nao Declarado,2019-07-01,2018,7+,2 Temporadas,"TV infantil, programas de TV coreanos, comédia...",Netflix
8803,s8805,Filmes,Zombieland,Estados Unidos,2019-11-01,2009,Restrito,88 min,"Comédias, Filmes de Terror",Netflix
8804,s8806,Filmes,Zoom,Estados Unidos,2020-01-11,2006,13+,88 min,"Filmes infantis e familiares, comédias",Netflix


**Salvando a 3° tabela em csv**


In [ ]:
amazon_netflix.to_csv('Amazon_netflix.csv',index=False)